In [540]:
import load_data
import pandas as pd
import sklearn.linear_model
import sklearn.ensemble
import matplotlib.pyplot as plt
import plot
import time
import datetime
import numpy as np

# allDf = load_data.getPandasDataframes()

bejAirDf = allDf[('Beijing', 'air')]
bejMetDf = allDf[('Beijing', 'met')]
bejGrd = allDf[('Beijing', 'grid')]

bejAirDf['newTime'] = pd.to_datetime(bejAirDf['time'], errors='coerce', format='%Y-%m-%d %H:%M:%S')

bejDf = pd.DataFrame()
for key in ['station_id', 'PM25_Concentration', 'PM10_Concentration', 'NO2_Concentration', 'CO_Concentration', 'O3_Concentration', 'SO2_Concentration']:
    bejDf[key] = bejAirDf[key]
bejDf['time'] = bejAirDf['newTime']

bejDf['hour'] = bejAirDf['newTime'].dt.hour
#bejDf['minute'] = bejAirDf['newTime'].dt.minute
#bejDf['second'] = bejAirDf['newTime'].dt.second
bejDf['day'] = bejAirDf['newTime'].dt.day
bejDf['month'] = bejAirDf['newTime'].dt.month
#bejDf['year'] = bejAirDf['newTime'].dt.year
bejDf['dayofweek'] = bejAirDf['newTime'].dt.dayofweek
bejDf['dayofweek'] = pd.Categorical(bejDf['dayofweek'], ordered=False)

valueDict = {(row['station_id'],row['time'].to_pydatetime()):row['PM10_Concentration'] for index, row in bejDf.iterrows()}
bejDf['PM10_3'] = bejDf['PM10_Concentration']
bejDf['PM10_3B'] = bejDf['PM10_Concentration']
bejDf['PM10_3C'] = bejDf['PM10_Concentration']
for index, row in bejDf.iterrows():
    rtime = row['time'].to_pydatetime()
    rtime = rtime - datetime.timedelta(hours=1)
    rtime2 = rtime - datetime.timedelta(hours=1)
    rtime3 = rtime2 - datetime.timedelta(hours=1)
    bejDf.at[index, 'PM10_3'] = valueDict.get((row['station_id'], rtime), np.NaN)
    bejDf.at[index, 'PM10_3B'] = valueDict.get((row['station_id'], rtime2), np.NaN)
    bejDf.at[index, 'PM10_3C'] = valueDict.get((row['station_id'], rtime3), np.NaN)


#stationDf = bejDf[bejDf['station_id']=='tiantan_aq']
#ToDo: limit to one station for now
#bejDf = stationDf

if False:
    plt.scatter(x=bejDf.PM25_Concentration, y=bejDf.PM10_Concentration)
    plt.xlabel("time", fontsize=14)
    plt.ylabel("PM10_Concentration", fontsize=14)
    f = plt.gcf()
    f.set_figheight(10)
    f.set_figwidth(15)
    plt.show()
    
bejDf = bejDf.fillna(bejDf.median())
# bejDf = pd.get_dummies(bejDf, 'dum')


In [541]:
# bejDf['businessHours'] = 0
# bejDf[(bejDf['hour'] >= 8) & (bejDf['hour']<=18)]['businessHours'] = 1
# bejDf.loc[workIndex,:]['businessHours'] = 1
# bejDf.head(n=100)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [542]:
ct = datetime.datetime.utcnow() - datetime.timedelta(days=2)
splitDate = pd.datetime(ct.year,ct.month,ct.day,  ct.hour)
trainDf = bejDf[bejDf['time'] < splitDate]
testDf = bejDf[bejDf['time'] >= splitDate]

In [543]:
cur = datetime.datetime.utcnow()
current_time = pd.datetime(cur.year,cur.month,cur.day,cur.hour)
d = list(testDf.time)
date_list = [current_time - datetime.timedelta(hours=x) for x in range(0, 48)]
date_list = [pd.to_datetime(i, format='%Y-%m-%d %H:%M:%S') for i in date_list]
missingDatesFromTestDf = np.setdiff1d(sorted(date_list),list(set(sorted(d))))
list(missingDatesFromTestDf)

[Timestamp('2018-04-28 05:00:00'),
 Timestamp('2018-04-29 00:00:00'),
 Timestamp('2018-04-29 16:00:00'),
 Timestamp('2018-04-29 17:00:00'),
 Timestamp('2018-04-29 23:00:00')]

In [539]:
targets = ['PM25_Concentration', 'PM10_Concentration', 'O3_Concentration']
features = [col for col in list(testDf) if ((col not in targets) and (col != 'time'))]

# print(trainDf[features].head(20))

lm = sklearn.linear_model.LinearRegression(n_jobs=-1)
#lm = sklearn.ensemble.RandomForestRegressor(n_jobs=-1, random_state=42)
lm.fit(trainDf[features], trainDf[targets])
print(lm.score(testDf[features], testDf[targets]))

#lm.fit(trainDf[features], trainDf['PM10_Concentration'])
#print(lm.score(testDf[features], testDf['PM10_Concentration']))

predictedValues = lm.predict(testDf[features])
predictedValues = pd.DataFrame(predictedValues, columns = targets)
actualValues = testDf[targets]

# plot.CreateMultiplePredictedAndActualValuesPlots([predictedValues['PM10_Concentration']],
#                                                  [actualValues['PM10_Concentration']], 
#                                                  'LinearRegression')

0.3009760225257948
